In [1]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('helper_functions/')
# import util as util
import util

In [2]:
from pymongo import MongoClient

client = MongoClient()
db = client.airbnb



In [25]:

db.listings_with_reviews_m.create_index('id')

# Create index on 'id' in the listings_with_calendar collection
db.listings_with_calendar.create_index('id')

'id_1'

In [26]:
db.listings_with_reviews_and_cal.drop()

pipeline = [
    {
        '$lookup': {
            'from': 'listings_with_calendar',
            'localField': 'id',
            'foreignField': 'id', 
            'as': 'cal_docs'
        }
    },
    {
        '$unwind': {
            'path': '$cal_docs',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$addFields': {
            'average_price': '$cal_docs.average_price',
            'first_available_date': '$cal_docs.first_available_date',
            'last_available_date': '$cal_docs.last_available_date',
            'dates_list': '$cal_docs.dates_list'
        }
    },
    {
        '$unset': 'cal_docs'
    },
    {
        '$out': 'listings_with_reviews_and_cal'
    }
]

# Execute the aggregation pipeline
db.listings_with_reviews_m.aggregate(pipeline)


In [3]:
print(db.listings_with_reviews_and_cal.count_documents({}))

39202


In [51]:
importlib.reload(util)


query = {'id': {'$regex': '^1000'}}
documents = list(db.listings_with_reviews_and_cal.find(query))

# Clean documents
documents = [util.clean_document(doc) for doc in documents]


# Check for datetime objects in cleaned documents
util.check_for_datetime(documents)



with open('listings_with_reviews_and_cal_subset_1000.json', 'w') as f:
    json.dump(documents, f, indent=4, cls=CustomJSONEncoder)

print("JSON file created successfully.")


JSON file created successfully.
